In [1]:
import numpy as np
import pandas as pd

# 数据框的分组与聚合

In [2]:
np.random.seed(2)
df = pd.DataFrame({
    "工号": np.arange(801, 809),
    "部门": np.random.choice(['A', 'B'], size=8),
    "年龄": np.random.randint(20, 50, size=8),
    "绩效": np.random.randint(60, 100, size=8),
})
df

,工号,部门,年龄,绩效
0,801,A,28,80
1,802,B,27,97
2,803,B,22,99
3,804,A,37,63
4,805,A,31,98
5,806,B,41,64
6,807,A,35,99
7,808,B,46,98


In [3]:
groups = [g for dept, g in df.groupby(["部门"])]
display(groups[0])
display(groups[1])

,工号,部门,年龄,绩效
0,801,A,28,80
3,804,A,37,63
4,805,A,31,98
6,807,A,35,99


,工号,部门,年龄,绩效
1,802,B,27,97
2,803,B,22,99
5,806,B,41,64
7,808,B,46,98


In [4]:
age1, kpi1 = groups[0].年龄.mean(), groups[0].绩效.mean()
age2, kpi2 = groups[1].年龄.mean(), groups[1].绩效.mean()
print("平均年龄：", age1, age2, " 平均绩效：", kpi1, kpi2)

平均年龄： 32.75 34.0  平均绩效： 85.0 89.5


In [5]:
pd.DataFrame({
    "年龄": [age1, age2], 
    "绩效": [kpi1, kpi2]
}, index=pd.Index(["A", "B"], name="部门"))

,年龄,绩效
部门,,
A,32.75,85.0
B,34.00,89.5


In [6]:
df.groupby(["部门"])[["年龄", "绩效"]].mean()

,年龄,绩效
部门,,
A,32.75,85.0
B,34.00,89.5


In [7]:
df.groupby(["部门"])[["年龄", "绩效"]].mean().reset_index()

,部门,年龄,绩效
0,A,32.75,85.0
1,B,34.00,89.5


<br>
参数`as_index`默认为`True`,将作为分组的列作为的数据框的行索引。

In [8]:
df.groupby(["部门"], as_index=False)[["年龄", "绩效"]].mean()

,部门,年龄,绩效
0,A,32.75,85.0
1,B,34.00,89.5


In [9]:
df.groupby(["部门"])[["年龄", "绩效"]].agg('mean')

,年龄,绩效
部门,,
A,32.75,85.0
B,34.00,89.5


In [10]:
df2 = df.groupby('部门').agg({'年龄':'median', '绩效': ['mean', 'sum', 'std']})
df2

年龄    绩效                
   median  mean  sum        std
部门                             
A    33.0  85.0  340  17.068489
B    34.0  89.5  358  17.019597

In [11]:
print(df2.to_latex())

\begin{tabular}{lrrrr}
\toprule
 & 年龄 & \multicolumn{3}{r}{绩效} \\
 & median & mean & sum & std \\
部门 &  &  &  &  \\
\midrule
A & 33.000000 & 85.000000 & 340 & 17.068489 \\
B & 34.000000 & 89.500000 & 358 & 17.019597 \\
\bottomrule
\end{tabular}



In [12]:
df2[[('绩效',   'mean'), ('绩效',    'sum')]]

绩效     
    mean  sum
部门           
A   85.0  340
B   89.5  358

## 2. 分组转换（transform）

In [13]:
df

,工号,部门,年龄,绩效
0,801,A,28,80
1,802,B,27,97
2,803,B,22,99
3,804,A,37,63
4,805,A,31,98
5,806,B,41,64
6,807,A,35,99
7,808,B,46,98


In [14]:
df.groupby('部门')['绩效'].transform('mean')

0    85.0
1    89.5
2    89.5
3    85.0
4    85.0
5    89.5
6    85.0
7    89.5
Name: 绩效, dtype: float64

In [15]:
df3 = df.copy()
df3['部门平均绩效'] = df3.groupby('部门')['绩效'].transform('mean')
df3['部门内绩效差异'] = df3.绩效 - df3.部门平均绩效
df3

,工号,部门,年龄,绩效,部门平均绩效,部门内绩效差异
0,801,A,28,80,85.0,-5.0
1,802,B,27,97,89.5,7.5
2,803,B,22,99,89.5,9.5
3,804,A,37,63,85.0,-22.0
4,805,A,31,98,85.0,13.0
5,806,B,41,64,89.5,-25.5
6,807,A,35,99,85.0,14.0
7,808,B,46,98,89.5,8.5


<br>

排名函数或秩函数[`rank()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rank.html)

In [16]:
pd.Series([30, 25, 10, 25, 40]).sort_values()

2    10
1    25
3    25
0    30
4    40
dtype: int64

In [17]:
pd.Series([30, 25, 10, 25, 40]).sort_values(ascending=False)

4    40
0    30
1    25
3    25
2    10
dtype: int64

In [18]:
df.sort_values(["部门", "绩效"], ascending=[True, False])

,工号,部门,年龄,绩效
6,807,A,35,99
4,805,A,31,98
0,801,A,28,80
3,804,A,37,63
2,803,B,22,99
7,808,B,46,98
1,802,B,27,97
5,806,B,41,64


In [19]:
pd.Series([10, 25, 25, 30, 40]).rank()

0    1.0
1    2.5
2    2.5
3    4.0
4    5.0
dtype: float64

In [20]:
pd.Series([10, 25, 25, 30, 40]).rank(method="first")

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
dtype: float64

In [21]:
pd.Series([10, 25, 25, 30, 40]).rank(method="average")

0    1.0
1    2.5
2    2.5
3    4.0
4    5.0
dtype: float64

In [22]:
pd.Series([10, 25, 25, 30, 40]).rank(method="min")

0    1.0
1    2.0
2    2.0
3    4.0
4    5.0
dtype: float64

In [23]:
pd.Series([10, 25, 25, 30, 40]).rank(method="max")

0    1.0
1    3.0
2    3.0
3    4.0
4    5.0
dtype: float64

In [24]:
df.groupby('部门').绩效.rank()

0    2.0
1    2.0
2    4.0
3    1.0
4    3.0
5    1.0
6    4.0
7    3.0
Name: 绩效, dtype: float64

In [25]:
df.groupby('部门').绩效.rank(ascending=False)

0    3.0
1    3.0
2    1.0
3    4.0
4    2.0
5    4.0
6    1.0
7    2.0
Name: 绩效, dtype: float64

In [26]:
df3['部门内绩效排名'] = df.groupby('部门').绩效.rank(method="max", ascending=False)
df3

,工号,部门,年龄,绩效,部门平均绩效,部门内绩效差异,部门内绩效排名
0,801,A,28,80,85.0,-5.0,3.0
1,802,B,27,97,89.5,7.5,3.0
2,803,B,22,99,89.5,9.5,1.0
3,804,A,37,63,85.0,-22.0,4.0
4,805,A,31,98,85.0,13.0,2.0
5,806,B,41,64,89.5,-25.5,4.0
6,807,A,35,99,85.0,14.0,1.0
7,808,B,46,98,89.5,8.5,2.0
